In [2]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

In [3]:
map = {}
class Data:
    # lay du lieu tu trong file
    def __init__(self, filename):
        lines = open(filename).read().split('\n')
        i = 0
        n = 1
        for line in lines:
            i = i + 1
            if i >= 7 and line != 'EOF' and line != '':
                type1 = [float(_) for _ in line.split()]
                map[n] = type1
                n = n + 1

In [63]:
class GA:
    def distance(self, city1, city2):
        if city1 in map and city2 in map:
            point1 = map[city1]
            point2 = map[city2]
            distance = np.power(((point1[1] - point2[1]) ** 2 + (point1[2] - point2[2]) ** 2), 0.5)
        return distance

    def routeDistance(self, route):
        distance = 0
        for i in range(0, len(route)):
            fromCity = route[i]
            toCity = None
            if i + 1 < len(route):
                toCity = route[i + 1]
            else:
                toCity = route[0]
            distance += self.distance(fromCity, toCity)
        return distance

    def createRoute(self):
        route = random.sample(list(map), len(map))
        return route

    def initialPopulation(self, popSize):
        population = {}
        popDistance = {}
        fitness = {}
        i = 0
        while i < popSize:
            child = self.createRoute()
            if child not in list(population.values()):
                population[i] = child
                popDistance[i] = self.routeDistance(population[i])
                fitness[i] = float(1/popDistance[i])
                i = i + 1
            else:
                i = i
        return population, popDistance, fitness

    def selection(self, population, fitness):
        # chon lua theo fitness = 1/distance, lua chon ngau nhien boi vong quay roullette
        # luu ti le phan tram tuong ung voi moi diem o percentFitness
        fitnessValue = 0
        percentFitness = {}
        for i in range(popSize):
            fitnessValue += fitness[i]
            percentFitness[i] = fitnessValue
        for i in fitness:
            percentFitness[i] = float(percentFitness[i] * 100 / fitnessValue)
        eliteRoute = {}
        i = 0
        while i < eliteSize:
            rand = float(random.random() * 100)
            for n in percentFitness:
                if percentFitness[n] <= rand < percentFitness[n + 1]:
                    if population[n] in list(eliteRoute.values()):
                        i = i
                    else:
                        eliteRoute[i] = population[n]
                        i = i + 1
        return eliteRoute

    def crossoverPop(self, eliteRoute, population):
        # lai ghep bang phuong phap doi
        i = 0
        children = {}
        child = []
        dulp = 0
        # lay phan chung cua parent 1 va parent 2, range [0, cutRange]
        while i < eliteSize:
            geneRange = []
            if i == eliteSize - 1:
                j = 0
            elif i != eliteSize - 1:
                j = i + 1
            for m in range(cutRange):
                geneRange.append(eliteRoute[i][m - 1])
            child = geneRange
            other = 0
            while other < len(map):
                for n in range(len(geneRange)):
                    if geneRange[n] == eliteRoute[j][other]:
                        dulp = dulp + 1
                if dulp == 0:
                    child.append(eliteRoute[j][other])
                else:
                    dulp = 0
                other = other + 1
                
            self.mutate(child)
            if child in list(population.values()):
                i = i
            else:
                children[i] = child
                i = i + 1
        return children

    def mutate(self, child):
        rand = float(random.randrange(0, 10000)) / 10000
        while rand <= mutationRate:
            print('Mutant!')
            swapped = 2
            swapWith = 5
            gene1 = child[swapped]
            gene2 = child[swapWith]
            child[swapped] = gene2
            child[swapWith] = gene1
            rand = float(random.randrange(0, 10000)) / 10000
        return child

    def evaluation(self, population, popDistance, fitness):
        eliteRoute = self.selection(population, fitness)
        children = self.crossoverPop(eliteRoute, population)
        i = 0
        worstFitness = {}
        while i < eliteSize:
            worstFitness[i] = 0
            for j in range(popSize):
                out = 0
                if fitness[j] <= fitness[worstFitness[i]]:
                    for n in range(len(worstFitness)):
                        if j == worstFitness[n]:
                            out = 1
                    if out == 0:
                        worstFitness[i] = j
            i = i + 1
        i = 0
        while i < eliteSize:
            population[worstFitness[i]] = children[i]
            popDistance[worstFitness[i]] = self.routeDistance(children[i])
            fitness[worstFitness[i]] = float(1/popDistance[worstFitness[i]])
            i = i + 1
        return population, popDistance, fitness

    def bestIndividual(self, population, popDistance):
        bestIndividual = {0: None, 1: None}
        bestDistance = 200000
        pos = 0
        for i in range(len(population)):
            if self.routeDistance(population[i]) <= bestDistance:
                bestDistance = popDistance[i]
                pos = i
        bestIndividual[0] = population[pos]
        bestIndividual[1] = bestDistance
        return bestIndividual

In [65]:
print ("File data: ")
# filename = raw_input ("")
filename = "dj38.tsp"
a = Data(filename)
popSize = 100
eliteSize = 50
cutRange = 5
mutationRate = 0.001
time = 5
#########

test = GA()
population, popDistance, fitness = test.initialPopulation(popSize)
BestDistance = popDistance[1]
i = 1
for i in range(time):
    population, popDistance, fitness = test.evaluation(population, popDistance, fitness)
    print ('Generation', i, 'best distance: ', test.bestIndividual(population, popDistance)[1])
    if BestDistance > test.bestIndividual(population, popDistance)[1]:
        BestDistance = test.bestIndividual(population, popDistance)[1]
        BestWay = test.bestIndividual(population, popDistance)[0]
print ('population:', population)
print('popDistance:', popDistance)
print('fitness:', fitness)
print('Best way:', BestWay)
print('Best distance:', BestDistance)
print('\n', popDistance)

File data: 
Generation 0 best distance:  22943.062482258698
Generation 1 best distance:  22943.062482258698
Generation 2 best distance:  22943.062482258698
Generation 3 best distance:  22534.92711452347
Mutant!
Generation 4 best distance:  21527.905229370634
population: {0: [10, 13, 15, 9, 4, 20, 3, 33, 34, 36, 16, 8, 17, 7, 37, 31, 2, 32, 18, 24, 19, 25, 35, 1, 22, 26, 6, 12, 11, 23, 28, 29, 14, 38, 21, 30, 27, 5], 1: [37, 20, 10, 2, 9, 8, 12, 21, 18, 31, 4, 5, 16, 7, 6, 1, 22, 24, 29, 17, 15, 28, 11, 19, 3, 25, 35, 36, 34, 23, 26, 13, 33, 14, 30, 27, 32, 38], 2: [5, 28, 12, 13, 31, 8, 14, 29, 32, 27, 24, 1, 4, 19, 33, 15, 10, 7, 9, 21, 2, 36, 34, 35, 37, 25, 18, 11, 22, 17, 3, 38, 6, 30, 23, 20, 26, 16], 3: [21, 15, 8, 25, 3, 2, 10, 38, 24, 23, 26, 35, 27, 32, 36, 5, 6, 7, 4, 37, 20, 17, 31, 33, 28, 34, 13, 11, 12, 1, 29, 19, 9, 18, 14, 16, 30, 22], 4: [15, 28, 12, 13, 31, 37, 23, 9, 16, 38, 22, 2, 6, 33, 34, 10, 17, 24, 3, 25, 1, 4, 8, 32, 18, 19, 27, 29, 21, 26, 35, 7, 20, 11, 5, 3